In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import urllib
import pandas as pd
import re
import numpy as np
from IPython.display import clear_output

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0'}
FQDN = 'https://www.y*******.**/'
responce = requests.get(FQDN)

print("request denied") if responce.status_code != 200 else print("ok")

In [ ]:
def get_html(url, params=None):
    r = requests.get(url, headers=header, params=params)
    return r

In [ ]:
url = 'https://www.y*******.**/search/'
response = get_html(url)
soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
title=[]
types_all=[]
deadline=[]
url=[]
description=[]

last_page = max([int(e.text) for e in soup.find_all('a', class_='page-numbers') if e.text != '»'])
page_links = [url + '?sf_paged=' + str(i+1) if i > 0 else url for i in range(last_page)]

for page in page_links[:]:
    response = get_html(page)
    soup = BeautifulSoup(response.text, 'lxml')
    links = [e.a.get('href') for e in soup.find_all('div', class_='post-header')]
    for link in links[:]:
        response = get_html(link)
        soup = BeautifulSoup(response.text, 'lxml')
        types = [e.text for e in soup.find_all('a', class_='meta-category')][0]
        types_all.append(types.replace('\n', ''))
        url.append(link)
        ddl = [e.li.text if e.li is not None else '' for e in soup.find_all('ul', class_='post-details')][0]
        deadline.append(ddl.replace('Deadline ', '')) if 'Deadline' in ddl else deadline.append('')
        p_tags=[e.find_all('p') for e in soup.find_all('div', class_='article-content')][0]
        description.append([p.text for p in p_tags[:5]])
        title.append([e for e in soup.find('h1', class_='page-title')][0])
    time.sleep(1)   

    clear_output(wait=True)
    print(page)
    print(len(title),len(deadline),len(description),len(url), len(types_all))

In [ ]:
df = pd.DataFrame()
df['course_name'] = title
df['course_type'] = types_all
df['url'] = url
df['deadline'] = deadline
df['description'] = description

descr = []
for idx,_ in df.iterrows():
    descr.append(', '.join(map(str, df.iloc[idx]['description'])))
df['description'] = descr

with pd.ExcelWriter('y******.xlsx') as writer:
    df.to_excel(writer)